In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/kaggle_1_train.csv")

In [3]:
df.head()

,Unnamed: 0,OUT,F1,F2,F3,INDEX
0,0,2.0100,16000,513,230,173613
1,1,4.6200,16000,1025,337,74258
2,2,1.0601,10000,513,107,4845
3,3,5.5900,20000,257,509,134428
4,4,3.4600,10000,257,347,15891


In [4]:
ranges = df.agg(["min", "max"])
ranges

,Unnamed: 0,OUT,F1,F2,F3,INDEX
min,0,0.10,8000,257,6,1
max,174286,43.88,24000,1025,7147,205044


In [6]:
stats = {
    "Mean": df.mean(),
    "Median": df.median(),
    "Mode": df.mode().iloc[0]
}

stats_df = pd.DataFrame(stats)
stats_df

,Mean,Median,Mode
Unnamed: 0,87143.000000,87143.00,0.0
OUT,5.449240,4.17,2.5
F1,16002.191787,16000.00,22000.0
F2,598.598214,513.00,513.0
F3,543.161016,365.00,121.0
INDEX,102556.511518,102594.00,1.0


In [7]:
train_df = pd.read_csv("/content/kaggle_1_train.csv")
test_df = pd.read_csv("/content/kaggle_1_test.csv")

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Simplified Dataset Class
class FeatureDataset(Dataset):
    def __init__(self, features, targets=None):
        self.features = torch.FloatTensor(features.values)
        self.targets = torch.FloatTensor(targets.values) if targets is not None else None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.targets is not None:
            return self.features[idx], self.targets[idx]
        return self.features[idx]

# Simplified Model Architecture
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze()

# Training Function
def train_model(train_df, test_df):
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        train_df[['F1', 'F2', 'F3']],
        train_df['OUT'],
        test_size=0.2,
        random_state=SEED
    )

    # Create datasets
    train_dataset = FeatureDataset(X_train, y_train)
    val_dataset = FeatureDataset(X_val, y_val)
    test_dataset = FeatureDataset(test_df[['F1', 'F2', 'F3']])

    # Create dataloaders
    batch_size = 256
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SimpleModel().to(device)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    best_val_loss = float('inf')
    epochs = 100

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.size(0)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                val_loss += criterion(outputs, y).item() * X.size(0)

        train_loss /= len(train_dataset)
        val_loss /= len(val_dataset)

        print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')

    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))

    # Generate predictions
    model.eval()
    test_preds = []
    with torch.no_grad():
        for X in test_loader:
            X = X.to(device)
            preds = model(X)
            test_preds.extend(preds.cpu().numpy())

    # Create submission
    submission = pd.DataFrame({
        'ID': test_df['INDEX'].values,
        'OUT': np.array(test_preds).flatten()
    })
    submission.to_csv('submission.csv', index=False)
    print('Submission file created!')

# Example usage:
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')
train_model(train_df, test_df)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Set seeds
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Enhanced Dataset Class with feature scaling
class FeatureDataset(Dataset):
    def __init__(self, features, targets=None):
        # Store original features for analysis
        self.features = features.values
        self.targets = targets.values if targets is not None else None

        # Manual scaling based on observed data ranges
        self.f1_scale = 24000  # max observed
        self.f2_scale = 1025   # max observed
        self.f3_scale = 7147   # max observed

        # Convert to tensors
        self.features = torch.FloatTensor(self._scale_features(self.features))
        if self.targets is not None:
            self.targets = torch.FloatTensor(self.targets)

    def _scale_features(self, features):
        """Manual feature scaling based on observed ranges"""
        scaled = features.copy()
        scaled[:, 0] = features[:, 0]/self.f1_scale    # F1 [8000-24000] → [0.33-1.0]
        scaled[:, 1] = features[:, 1]/self.f2_scale    # F2 [257-1025] → [0.25-1.0]
        scaled[:, 2] = np.log1p(features[:, 2])/np.log1p(self.f3_scale)  # F3 log-scale
        return scaled

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.targets is not None:
            return self.features[idx], self.targets[idx]
        return self.features[idx]

# Enhanced Model Architecture
class EnhancedModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Feature-specific initial layers
        self.f1_path = nn.Sequential(
            nn.Linear(1, 64),
            nn.BatchNorm1d(64),
            nn.SiLU(),
            nn.Dropout(0.2)
        )

        self.f2_path = nn.Sequential(
            nn.Linear(1, 64),
            nn.BatchNorm1d(64),
            nn.SiLU()
        )

        # Special processing for F3 (high variance)
        self.f3_path = nn.Sequential(
            nn.Linear(1, 128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.3)
        )

        # Main network
        self.main_net = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.SiLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # Split features
        f1 = x[:, 0:1]
        f2 = x[:, 1:2]
        f3 = x[:, 2:3]

        # Process each feature separately
        f1_out = self.f1_path(f1)
        f2_out = self.f2_path(f2)
        f3_out = self.f3_path(f3)

        # Combine features
        combined = torch.cat([f1_out, f2_out, f3_out], dim=1)
        return self.main_net(combined).squeeze()

# Asymmetric loss function
class WeightedMSELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, target):
        weights = torch.where(target > 5.0, 1.5, 1.0)  # Higher weight for large values
        return (weights * (pred - target)**2).mean()

def train_model(train_df, test_df):
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        train_df[['F1', 'F2', 'F3']],
        train_df['OUT'],
        test_size=0.2,
        random_state=SEED
    )

    # Create datasets
    train_dataset = FeatureDataset(X_train, y_train)
    val_dataset = FeatureDataset(X_val, y_val)
    test_dataset = FeatureDataset(test_df[['F1', 'F2', 'F3']])

    # Create dataloaders
    batch_size = 256
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = EnhancedModel().to(device)

    # Loss and optimizer
    criterion = WeightedMSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

    # Training loop
    best_val_loss = float('inf')
    epochs = 100

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item() * X.size(0)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                val_loss += criterion(outputs, y).item() * X.size(0)

        train_loss /= len(train_dataset)
        val_loss /= len(val_dataset)

        # Update scheduler
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')

    # Load best model and predict
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    test_preds = []
    with torch.no_grad():
        for X in test_loader:
            X = X.to(device)
            preds = model(X)
            test_preds.extend(preds.cpu().numpy())

    # Create submission
    submission = pd.DataFrame({
        'ID': test_df['INDEX'].values,
        'OUT': np.array(test_preds).flatten()
    })
    submission.to_csv('submission.csv', index=False)
    print('Submission file created with enhanced model!')

# Usage:
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')
train_model(train_df, test_df)